In [ ]:
import os
import random
import time
from azure.iot.device import IoTHubDeviceClient, Message

import requests
import pandas as pd
import numpy as np
import json
from pathlib import Path
import datetime

import finnhub
import yfinance as yf

BBY_purchase_price=70
C_purchase_price=60

##Creating Configurations for API and IoTHub. config file has the confidential keys
def load_config(config_path):
        config = json.loads(Path(config_path).read_text())
        API_KEY = config.get('API_KEY')
        CONNECTION_STRING = config.get('CONNECTION_STRING')
        return API_KEY, CONNECTION_STRING

# Setup client


TIMEINTERVAL = 60

###Function for data capture and sending messages
def run_telemetry_sample(client, API_KEY):
    # This sample will send stock telemetry every second
    print("IoT Hub device sending periodic messages")

    client.connect()
    print('Connected with the client')
    iMsgCnt=0

    
    while True:
       iMsgCnt +=1 
       ######Finnhub data collection
       finnhub_client = finnhub.Client(api_key=API_KEY)
       res1 = finnhub_client.quote('C')
       res2 = finnhub_client.quote('BBY')

       print(res1['t'])
       t = res1['t']
       citi = res1['c']
       bestbuy = res2['c']
       
       return_citi_finnhub=(citi-C_purchase_price)/C_purchase_price*100
       return_bestbuy_finnhub=(bestbuy-BBY_purchase_price)/BBY_purchase_price*100
    # Get the data of the stock

    ### yfinance

       ticker_symbol = 'C,BBY'
       citi_bby_stock = yf.Tickers(ticker_symbol)
       historical_prices = citi_bby_stock.history(period='1d', interval='1m')
       latest_price = historical_prices['Close'].iloc[-1]
       latest_time = historical_prices.index[-1]
       citi_yfinance=latest_price['C']
       bby_yfinance=latest_price['BBY']
       
       ############Anomaly Detection in Time Series of C and BBY##############
       ticker_symbol = 'C,BBY'
       citi_bby_stock = yf.Tickers(ticker_symbol)
       historical_prices_14 = citi_bby_stock.history(period='14d', interval='1h')
       q1 = np.percentile(historical_prices_14['Close']['BBY'], 25)
       q3 = np.percentile(historical_prices_14['Close']['BBY'], 75)
       iqrBBY = q3 - q1
       low_lim_bby = q1 - 1.5 * iqrBBY
       up_lim_bby = q3 + 1.5 * iqrBBY
       #historical_prices_14['Close']['C']
       q1 = np.percentile(historical_prices_14['Close']['C'], 25)
       q3 = np.percentile(historical_prices_14['Close']['C'], 75)
       iqrC = q3 - q1
       low_lim_c = q1 - 1.5 * iqrC
       up_lim_c = q3 + 1.5 * iqrC

       if (citi_yfinance<low_lim_c) or (citi_yfinance>up_lim_c):
           anomaly_citi=1
       else:
           anomaly_citi=0
           
       if (bby_yfinance<low_lim_bby) or (bby_yfinance>up_lim_bby):
           anomaly_bby=1
       else:
           anomaly_bby=0



       return_citi_yfinance=(citi_yfinance-C_purchase_price)/C_purchase_price*100
       return_bestbuy_yfinance=(bby_yfinance-BBY_purchase_price)/BBY_purchase_price*100

       max_bby_today=max(historical_prices['Close']['BBY'])
       min_bby_today=min(historical_prices['Close']['BBY'])

       max_c_today=max(historical_prices['Close']['C'])
       min_c_today=min(historical_prices['Close']['C'])


       k=historical_prices['Close'].dropna()
       correlation=np.corrcoef(k['BBY'],k['C'])    


       sMsgTxt = f'{{"message":{iMsgCnt},"timestamp":{t},"Citi Group":{citi},"Best Buy":{bestbuy}, "Citi Group Yfinance":{citi_yfinance},"BBY Yfinance":{bby_yfinance},"Return on Citi Finnhub": {return_citi_finnhub},"Return on BBY Finnhub":{return_bestbuy_finnhub},"Return on Citi YFinance":{return_citi_yfinance},"Return on BBY YFinance":{return_bestbuy_yfinance},"Maximum BBY Today":{max_bby_today},"Maximum C Today":{max_c_today},"Minimum BBY Today":{min_bby_today},"Minimum C Today":{min_c_today},"Anomaly C":{anomaly_citi},"Anomaly BBY":{anomaly_bby},"correlation":{correlation[0][1]}}}'

       message = Message(sMsgTxt)

       print("Sending message: {}".format(message))
       client.send_message(message)
       print("Message successfully sent")
       time.sleep(TIMEINTERVAL)


# def main():
if __name__ == "__main__":
    print("IoT Hub Quickstart")
    print("Press Ctrl-C to exit")
    

    # Instantiate the client. Use the same instance of the client for the duration of
    # your application
    API_KEY, CONNECTION_STRING = load_config('./config.json')
    
    client = IoTHubDeviceClient.create_from_connection_string(CONNECTION_STRING)
    

# #Convert to Pandas Dataframe

    
    # Run Sample
    try:
        run_telemetry_sample(client, API_KEY)
    except KeyboardInterrupt:
        print("IoTHubClient sample stopped by user")
    finally:
        # Upon application exit, shut down the client
        print("Shutting down IoTHubClient")
        client.shutdown()

